<h3> Useful links for this notebook:<br><br>
For converting the data:<br>
[1] https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial <br>
[2] https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html <br>
[3] http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/ <br><br>

Data generators:<br>
[4] https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html <br>
[5] https://keras.io/utils/#sequence <br>

In [17]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import constants
import IPython.display as ipd  # To play sound in the notebook
np.random.seed(1001)
import os
from os import listdir
from os.path import isfile, join
import shutil
import IPython
import matplotlib
import seaborn as sns
from scipy.io import wavfile
from sklearn import preprocessing
import sys

In [2]:
data_loc="database/"
wav_path = data_loc+"audio_train/"
sr=44100
dataset = pd.read_csv(data_loc+"train.csv",header=0,names=['fname','label','verified'])
checkedDataset = dataset.loc[dataset.verified == 1,["fname","label"]]

In [3]:
def load_wav_file(name):
    _, b = wavfile.read(wav_path + name)
    assert _ == sr
    return b

checkedDataset['time_series'] = checkedDataset['fname'].apply(load_wav_file)
checkedDataset['nframes'] = checkedDataset['time_series'].apply(len)
checkedDataset=checkedDataset.reset_index()

In [4]:
# First, lets normalize the time series
# Perhaps we should address the warning given below?
for i in range(checkedDataset.shape[0]):
    c=checkedDataset.time_series[i]
#     checkedDataset.time_series[i] = c/np.linalg.norm(c)
    npmin=np.min(c)
    npmax=np.max(c)
    checkedDataset.time_series[i] = (c-npmin) / (npmax - npmin)

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in short_scalars
  import sys


In [5]:
def split(x,y,keep):
    # test/train split
    # trainsz - size of train set to retain; eg 0.7
    trsize=int(x.shape[0] * keep)

    x_train=x[:trsize]
    x_val=x[trsize:] 

    y_train=y[:trsize] 
    y_val=y[trsize:] 
    
    return x_train,x_val,y_train,y_val

def encode(data, oneHot=False):
    # Encode labels into integers or onehot
    le = preprocessing.LabelEncoder()
    le.fit(data)
    trans = le.transform(data)
    if oneHot== True:
        ohlen = (len(set(trans)))
        eye=np.eye(ohlen)
        return np.array([eye[i] for i in trans])
    else:
        return trans

In [6]:
y=encode(checkedDataset.label.values, oneHot=True) # Get labels

In [7]:
xtr,xtst,ytr,ytst = split(checkedDataset.time_series.values,y, 0.8) # Split data into test/train
xval, xtst, yval, ytst = split(xtst, ytst, 0.5) # Split train into train/val

In [19]:
print ('Xtr shape: {}, Xtr[0] shape: {}. Note here that each row of Xtr is a list containing a time series!'.format(\
                                                                                                    xtr.shape, xtr[0].shape))
print (xtr.shape[0] + xval.shape[0] + xtst.shape[0])
print (y.shape)

Xtr shape: (2968,), Xtr[0] shape: (455112,). Note here that each row of Xtr is a list containing a time series!
3710
(3710, 41)


In [9]:
from keras.utils import Sequence

class DataGen(Sequence):
    """
    See [4], [5] to understand how data generators work.
    CURRENTLY: This generator does work, but need to fix all MFCC getting related functions
    to properly deal with our data format (and finish some of them)
    """
    def __init__(self, x, labels, batch_size=2):
        self.x, self.y= x, labels
        self.batch_size=batch_size
        
    def frame(self, audio, FFT_size=2048, hop_size=10, sample_rate=44100):
        """     
        Convert batch of time series to batch of audio frame
        Not currently working fully for batches, there is a cell below for
        playing with this.
        Taken from https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
        hop_size in ms
        """
        audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
        frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
        frame_num = int((len(audio) - FFT_size) / frame_len) + 1
        frames = np.zeros((frame_num,FFT_size))

        for n in range(frame_num):
            frames[n] = audio[n*frame_len:n*frame_len+FFT_size]

        return frames
    
    def freq_domain(self):
        # Convert audio frame into frequency domain
        pass
    
    def mfcc(self):
        # Given frequency domains, get the MFCCs
        pass

    def get_mfcc(self, batch):
        """ 
        Given a batch of data X, we have n rows. Each row contains
        a time series. Each of these time series needs to be converted to an audio frame and then
        to a frequency domain. We can then use that to filter and extract useful MFCC features. 
        See [1], [2], [3].
        """
        # Convert batch to MFCCs
        mfccs=[]
        for series in batch:
            fr=self.frame(series[0])
            print(fr.shape)
            mfccs.append(fr)
        return np.array(mfccs)
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self,idx):
        tmp_x=self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
#         batch_x=self.mfcc(tmp_x)
        
        return tmp_x, batch_y

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
data=DataGen(xtr, ytr)

In [12]:
data[0][0]

array([array([0.44391037, 0.44394571, 0.44387503, ..., 0.44504135, 0.44504135,
       0.44504135]),
       array([0.51426635, 0.5148826 , 0.51666975, ..., 0.51426635, 0.51426635,
       0.51426635])], dtype=object)

In [13]:
# Need to find out how to convert batch of time series to batch of frames

def frame(audio, FFT_size=2048, hop_size=10, sample_rate=44100):
    # Taken from https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
    # hop_size in ms
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    frames = np.zeros((frame_num,FFT_size))

    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]

    return frames

frt=frame(data[0][0][0])

In [20]:
print (frt.shape)

(1033, 2048)
